In [ ]:
from typing import Dict, Union
from snowflake.snowpark import Session

connection_parameters: Dict[str, Union[int, str]] = {
    "account": "<account-identifier>",
    "user": "<username>",
    "password": "<password>",
    "role": "ACCOUNTADMIN"
}

In [ ]:
session = Session.builder.configs(connection_parameters).create()

In [ ]:
df = session.sql('SELECT * FROM snowflake_sample_data.tpch_sf10.orders')

df.show()

In [ ]:
print(session.get_current_account())
print(session.get_current_database())
print(session.get_current_role())
print(session.get_current_schema())

In [ ]:
session.close()